In [212]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt 
import sys, os 
import time
import csv
%matplotlib inline

In [213]:
# constants 
EPS0 = 8.85 * 10 ** -12
QE = 1.602 * 10 ** -19 
K = 1.381 * 10 ** -23 
AMU = 1.661 * 10 ** -27
MI = 40 * AMU #mass of an Ar+ ion 

#simulation inputs 
N = 2000 #max particles 
n0 = 1 * 10 ** 12 #density
V_ref = 0 #reference potential 
Te = 1  #electron temperature in eV 
Ti = 0.1 #ion temperature in eV
V_d = -5.1 #dust potential (use values from lit)
v_drift = 7000
tol = 1 * 10 ** -1
iterations = 5


#plasma parameters 
lambd_D = np.sqrt( EPS0 * Te /(n0 * QE) )
vth = np.sqrt((2 * QE * Ti) / MI)
#v0 = 0.2 (stream velocity)  


#domain parameters 
nx = 10
ny = 10
J = nx * ny 
dx = dy = dh = lambd_D
Lx = (nx-1) * dx 
Ly = (ny-1) * dy 
dt = (0.1 * dx) / v_drift 
np_in = (ny-1) * 15 

#specific weight 
flux = n0 * v_drift * Ly 
npt = flux * dt #particles created per timestep 
sw = npt / np_in #specific weight 
q_mp = 1 #macroparticle charge 

# particle arrays
p_pos = np.zeros([N,2])
p_velo = np.zeros([N,2])

#q = -1.6 * 10 ** -19

#rho = (sw + (q_mp * q)) / (dh ** 2)

#print((1-hx) * (1-hy))

In [214]:
start = time.clock() # Keeping track of computational time 

def charge_depo(dh, NP, J):

    q = np.zeros((nx, ny))
    rho = np.zeros((ny, nx))

    for p in range (1, NP): 
            
            fi = (1 + p_pos[p,0]) / (dh) 
            i = np.floor(fi)
            hx = fi - i  
            #print(hx)    
            
            fj = (1 + p_pos[p,1]) / (dh)
            j = np.floor(fj)
            hy = fj - j 
            #print(1. - int(hy))            
            
            
            q[i,j] = q[i,j] + (1-hx) * (1-hy)
            q[i+1, j] = q[i+1, j] + hx * (1-hy)
            q[i, j+1] = q[i, j+1] + (1-hx) * hy 
            q[i+1, j+1] = q[i+1, j+1] + hx * hy 
            
            
            rho = (sw + q_mp * q) / (dh * dh)
                
            rho[0,:] = 2 * rho[0,:]
            rho[-1, :] = 2 * rho[-1, :]
            rho[:, 0] = 2 * rho[:, 0]
            rho[:, -1] = 2 * rho[:, -1]
            
            rho = rho + (1 * 10 ** 4)
            #print(rho)
    return q, rho 

print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0010259999999995273 seconds


In [215]:
start = time.clock() # Keeping track of computational time 

#stencil array used in potential solver 

def stencil_fd(J, dx, dh, nx, ny):
    """Uses the finite difference scheme"""
    #E = []
    
    #source = rho
    M = np.zeros((J,J))
    
    for i in range (1, nx-1):
        for j in range(1, ny-1): 
            u = (j-1) * nx + i
            M[u, u] = -4. / (dh * dh) 
            M[u, u-1] = 1. / (dh * dh)
            M[u, u+1] = 1. / (dh * dh)
            M[u, u-nx] = 1. / (dh * dh)
            M[u, u+ (nx-2)] = 1. / (dh * dh)    
    
    for i in range (0, nx):
        u = i+1
        M[u,u] = -1. / dh
        M[u,u+nx] = 1. / dh 
        
    for i in range (0,nx): 
        u = (ny-1) * nx  + i 
        M[u, u-nx] = 1. / dh
        M[u, u] = -1. / dh 
        
    for j in range (0, ny): 
        u = (j-1) * nx + nx 
        M[u, :] = np.zeros([1, J])
        M[u, u-1] = 1. / dh 
        M[u, u] = -1. / dh 
        
    for j in range(0, ny): 
        u = (j-1) * nx + 1 
        M[u, :] = np.zeros([1, J])
        M[u, u] = 1.     
        
    return M
print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0004989999999978068 seconds


In [216]:
start = time.clock() # Keeping track of computational time 

def solver_2d(rho, tol, Ti, n0, V_ref, QE, iterations): 
    'Uses Gauss Sidel method'
        
    M = stencil_fd(J, dx, dh, nx, ny)
    V = np.ones((ny,nx)) #* V_ref
    
    source = rho
    NX = NY = np.size(source,1)
    NN = np.size(source)
    
    b0 = np.reshape(source, (NN,1))
    x = np.reshape(V, (np.size(V), 1))
    
    #print(np.size(x), np.size(b0))
    
    b = b0 - n0 * np.exp((x-V_ref)/ Te)
    b = b * QE / EPS0     
    
    for it in range (0, iterations):
        
        b[0:NX] = 0 
        b[NN-NX+1 : NN] = 0 
        b[NX: NX : NN] = 0 
        b[1:NX:NN] = V_ref
        
        if it % 10 == 0:
            R = np.linalg.norm(b-M*x)
            #print(b-M*x)
            if R <= tol:
                print('Solver converged!')
                break 
            #if R > tol: 
                #print('Solver failed to converge. Check R!')
    
    V = np.reshape(x, (NX,NY))
    return V
print("Clocking in at %s seconds"  % (time.clock() - start))

Clocking in at 0.0003849999999978593 seconds


In [217]:
NP = 1

for it in range (1, iterations):
    q, rho = charge_depo(dh, NP, J)

    V = solver_2d(rho, tol, Ti, n0, V_ref, QE, iterations)   
    
     #E field solver
    Ex = np.zeros([nx, ny])
    Ey = np.zeros([nx, ny])
    E = np.zeros([nx, ny])
    
    #internal nodes 
    Ex[1:nx-1, :] = V[0:nx-2,:] - V[nx-(nx-2):, :]
    Ey[0: ,1:nx-1] = V[:, 0:ny-2] - V[:, 2:ny]

    #boundaries
    #multiplied by 2 to keep values equivalent to internal nodes
    Ex[0,:] = 2* (V[0,:] - V[1,:]) 
    Ex[nx-1, :] = 2 * (V[nx-2,:] - V[nx-1, :])
    
    Ey[:, 0] = 2 * (V[:,0] - V[:,1])
    Ey[:,ny-1] = 2 * (V[:, ny-2] - V[:, ny-1])
    
    Ex = np.floor (Ex / (2 * dx) )                                       
    Ey = Ey / (2 * dy)
    
    #generate particles 
    
    if NP + np_in >= N:
        np_in = N - NP
   
    #insert paritcles 
    #(NOTE: save this for after 2d environment works)
    
    p_pos[NP:NP+np_in, 1:] = np.random.rand(np_in,1) * dh #x position 
    p_pos[NP:NP+np_in, 1:] = np.random.rand(np_in,1) * Ly #y position 
    
    #sample Maxwellian in x,y 
    #add drift velocity in x 
    
    p_velo[NP:NP+np_in, 1:] = v_drift + (-1.5 + np.random.rand(np_in,1) + np.random.rand(np_in,1) 
                                         + np.random.rand(np_in, 1)) * vth 
    p_velo[NP:NP+np_in, 1:] = 0.5 * (-1.5 + np.random.rand(np_in,1) + np.random.rand(np_in,1) 
                                     + np.random.rand(np_in, 1)) * vth
    
    #move particles 
    p = 1 
    
    while p <= NP:
            
        fi = 1 + p_pos[p,0]/dx 
        i = np.floor(fi)
        hx = fi - i  
        
        fj = 1+ p_pos[p,1]/dy 
        j = np.floor(fj)
        hy = fj - j 
            
        E = ([Ex[i, j], Ey[i,j]]) * (1-hx) * (1-hy)                                                
        E = E + ([Ex[i + 1, j], Ey[i + 1, j]]) * hx * (1-hy)                                                
        E = E + ([Ex[i, j + 1], Ey[i + 1, j]]) * (1-hx) * hy                                                
        E = E + ([Ex[i + 1, j + 1], Ey[i + 1, j + 1]]) * hx * hy  
        
        F = QE * E 
        a = F/MI 
        
        p_pos[p, :] = p_velo[p, :] + a * dt 
        p_velo[p, :] = p_pos[p, :] + p_velo[p, :] * dt 
        
        if p_pos[p,1] < 0: 
            p_pos[p,1] = -p_pos[p,1]
            p_velo[p,1] = -p_velo[p,1]
            
        p = p + 1 
        print(p_pos, p_velo)

<class 'list'> <class 'float'>


/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:60: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:61: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:62: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Isabel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


TypeError: can't multiply sequence by non-int of type 'float'

In [241]:
a = np.arange(5) - 4
b = a.reshape((5, 1)) * 0 
c = b * 0.00002 
d = [0, -17967.60810524, -17967.60810524, -17967.60810524, 0] 
R = np.linalg.norm(d-b)

print(b, d, R)

[[0]
 [0]
 [0]
 [0]
 [0]] [0, -17967.60810524, -17967.60810524, -17967.60810524, 0] 69588.2469628


In [229]:
V = np.ones((ny,nx)) #* V_ref
    
source = rho
NX = NY = np.size(source,1)
NN = np.size(source)
b0 = np.reshape(source, (NN,1))
x = np.reshape(V, (np.size(V), 1))


B = b0 - n0 * np.exp((x-V_ref)/ Ti)
B = (b * QE) / EPS0     # around -17xxx.xxxxx
r = np.linalg.norm(B)

NameError: name 'b0' is not defined

In [ ]:
#start = time.clock() # Keeping track of computational time 
#
#def E_from_V(rho, J, dx):
#    """Uses the finite difference scheme"""
#    #E = []
#    source = rho[0:J-1]*dx**2
#    #M = np.zeros((J-1,J-1))
#    
#    M = np.zeros((J,J))
#    
#    for i in range (1, nx-1):
#        for j in range(1, ny-1): 
#            u = (j-1) * nx + i
#            M[u, u] = -4. / (dh * dh) 
#            M[u, u-1] = 1. / (dh * dh)
#            M[u, u+1] = 1. / (dh * dh)
#            M[u, u-nx] = 1. / (dh * dh)
#            M[u, u+ (nx-2)] = 1. / (dh * dh)    
#    
#    for i in range (0, nx):
#        u = i+1
#        M[u,u] = -1. / dh
#        M[u,u+nx] = 1. / dh 
#        
#    for i in range (0,nx): 
#        u = (ny-1) * nx  + i 
#        M[u, u-nx] = 1. / dh
#        M[u, u] = -1. / dh 
#        
#    for j in range (0, ny): 
#        u = (j-1) * nx + nx 
#        M[u, :] = np.zeros([1, J])
#        M[u, u-1] = 1. / dh 
#        M[u, u] = -1. / dh 
#        
#    for j in range(0, ny): 
#        u = (j-1) * nx + 1 
#        M[u, :] = np.zeros([1, J])
#        M[u, u] = 1.     
#
#    for i in range(0, J-1):
#        for j in range(0, J-1):
#            if i == j: 
#                M[i,j] = 2.
#            if i == j-1:
#                M[i,j] = -1.
#            if i == j+1: 
#                M[i,j] = -1.
#                
#    M[0, J-2] = -1.
#    M[J-2, 0] = -1.
#        
#    V = np.linalg.solve(M, source)
#        
#    E = np.zeros((J,2))
#    
#    for i in range (1,J-2): 
#        E[i] = (V[i+1] - V[i-1]) / 2./dx
#    E[J-2] = (V[0] - V[J-3]) / 2./dx
#    E[0] = (V[1] - V[J-2]) / 2./dx
#    E[J-1] = E[0]
#    E = -E
#    #E = np.append(i)        
#    return E
#
#
#print("Clocking in at %s seconds"  % (time.clock() - start))